**更新配点**

In [1]:
%reset -f

In [2]:
import sys, os
sys.path.append(os.path.dirname(os.path.realpath('../utils')))
import numpy as np
from matplotlib import pyplot as plt
from utils.aiwt import cjkfun, z_g, CompressInterp, adapt_CjJ4
from utils.pde import theta1_2d, Precise

In [3]:
J = 15
j0 = 0

a = 0
b = 2
Re = 1000

deltax = (b-a)/(2**J)
XJ0 = np.arange(a, b+deltax, deltax).reshape(1,-1)
V0 = np.sin(np.pi * XJ0)

M = 1
CC = 1

t = 0.4
tao = 0.001

eps0 = 0.005


In [4]:
cjk, cjknum, zJc, zJcnum = cjkfun(V0, J, j0, a, b, eps0)
gjk, gjknum, gJc, gJcnum = z_g(j0, J, cjk, cjknum, zJc, zJcnum, a, b, M, CC)
XJ = a + deltax * gJc
V0 = CompressInterp(XJ0, V0, XJ)
C = adapt_CjJ4(J, j0, gjk, gjknum, gJc, gJcnum, a, b)
theta1d, theta2d = theta1_2d(J, j0, a, b, C, gjk, gjknum, gJc, gJcnum)
V1 = Precise(V0, Re, theta1d,theta2d,gJcnum,tao)

In [5]:
def cjkfun2(VJc, J, C, j0, gjk, gjknum, gJcnum, eps0):
    """
    输入：
        VJc : ndarray(1, n) -- 待插值函数
        J : int -- 小波插值层数
        C : list[ndarray()] -- 多尺度插值算子
        j0 : int -- 初始层数
        gjk : list[ndarray(-1), ...] -- 前一层每层所有配点位置索引
        gjknum : list -- 层配点索引数量（每层需计算的配点数量）
        gJcnum : int -- 配点实际位置的数量
        eps0 : float -- 阈值
    功能：
        在上一个空间中获取到的配点的基础上，再次计算得到新空间的配点
    输出：
        cjk : list[ndarray(-1), ...] -- 的配点位置索引
        cjknum : list -- 层配点索引数量（每层需计算的配点数量）
        zJc : list -- 配点实际位置
        zJcnum : int -- 实际位置的数量
    """
    numJ = 2**j0 + 1
    
    cjk = [np.array([0])]
    cjknum = []
    zJc0 = np.arange(numJ) * 2**(J-j0)
    for j in range(j0, J):
        num = 0
        if gjknum[j] > 0:
            # 配点代表的 “权值” 与函数空间之间的“差异”
            cjkvalue = np.dot(C[j-1], VJc)
            # 阈值
            kt = np.flatnonzero((abs(cjkvalue) > eps0))
            num = kt.shape[0]
            if num > 0:
                kk = gjk[j][kt]
                cjk.append(kk)
                zJc0 = np.append(zJc0, (2 * kk + 1) * (2 ** (J - j - 1)), 0)
        cjknum.append(num)
    zJc = np.sort(zJc0).tolist()
    zJcnum = len(zJc)
    return cjk, cjknum, zJc, zJcnum

In [6]:
cjk, cjknum, zJc, zJcnum = cjkfun2(V1, J, C, j0, gjk, gjknum, gJcnum, eps0)